In [ ]:
import os
import numpy as np
from scipy.linalg import expm
import matplotlib.pyplot as plt
from im import coupled_dynamics
from constants import sigmax, sigmay, sigmaz
from cli_utils import _hdf2im, _hdf2trained_im

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica",
    "font.size": 20,
})
script_dir = os.path.dirname(os.path.abspath(''))

# ----------------------------------------------------------------------------------- #

plotting_config = {
    "left_ims":  # left im's stumps
        [
            "J_0.1_do/2023-09-01_11:07:27+0200",
            "J_0.1_it/2023-09-28_17:24:16+0200",
            "J_0.1_do_xy/2023-10-04_11:33:29+0200",
            "J_0.1_it_xy/2023-10-04_11:34:39+0200",
        ],
    "right_ims":  # right im's stumps
        [
            "J_0.1_up/2023-08-31_22:30:38+0200",
            "J_0.1_up/2023-08-31_22:30:38+0200",
            "J_0.1_up_xy/2023-10-04_11:33:54+0200",
            "J_0.1_up_xy/2023-10-04_11:33:54+0200",
        ],
    "names": # figures' names
        [
            "appx_xxx_relaxation_du",
            "appx_xxx_relaxation_iu",
            "appx_xx_relaxation_du",
            "appx_xx_relaxation_iu",
        ],
    "xx": [0.1,  0.1,  0.1, -0.1],  # XX interaction amplitudes between impurity's spins
    "yy": [0.1,  0.1, -0.1,  0.1],  # YY interaction amplitudes between impurity's spins
    "zz": [0.1, -0.1, -0.1,  0.1],  # ZZ interaction amplitudes between impurity's spins
    "xl": [0.0,  0.0,  0.0,  0.0],  # X fields applied to the left spin
    "xr": [0.0,  0.0,  0.0,  0.0],  # X fields applied to the right spin
    "yl": [0.0,  0.0,  0.0,  0.0],  # Y fields applied to the left spin
    "yr": [0.0,  0.0,  0.0,  0.0],  # Y fields applied to the right spin
    "zl": [0.0,  0.0,  0.0,  0.0],  # Z fields applied to the left spin
    "zr": [0.0,  0.0,  0.0,  0.0],  # Z fields applied to the right spin
    "dens":  # initial impurity's density matrix
        np.array([
            1., 0., 0., 0.,
            0., 0., 0., 0.,
            0., 0., 0., 0.,
            0., 0., 0., 0., 
        ]),
}

# ----------------------------------------------------------------------------------- #

left_ims_path = [f"{script_dir}/experiments/output/physical_im/{left_im}" for left_im in plotting_config['left_ims']]
right_ims_path = [f"{script_dir}/experiments/output/physical_im/{right_im}" for right_im in plotting_config['right_ims']]

xx_list = list(map(float, plotting_config["xx"]))
yy_list = list(map(float, plotting_config["yy"]))
zz_list = list(map(float, plotting_config["zz"]))
xl_list = list(map(float, plotting_config["xl"]))
xr_list = list(map(float, plotting_config["xr"]))
yl_list = list(map(float, plotting_config["yl"]))
yr_list = list(map(float, plotting_config["yr"]))
zl_list = list(map(float, plotting_config["zl"]))
zr_list = list(map(float, plotting_config["zr"]))
rows_number = int(len(xx_list) / 2)
figures = []
for left_im_path, right_im_path in zip(left_ims_path, right_ims_path):
    im_exact1 = _hdf2im(left_im_path)
    im_exact2 = _hdf2im(right_im_path)
    im_trained1 = list(reversed(_hdf2trained_im(left_im_path)))
    im_trained2 = list(reversed(_hdf2trained_im(right_im_path)))
    fig = plt.figure(figsize=(2 * 6.4 / 1.5, rows_number * 4.8 / 1.5))
    for k, (xx, yy, zz, xl, xr, yl, yr, zl, zr) \
        in enumerate(zip(xx_list, yy_list, zz_list, xl_list, xr_list, yl_list, yr_list, zl_list, zr_list)):
        rho_in = np.array([np.array(float(x), dtype=np.complex128) for x in plotting_config["dens"]])
        h = xx * np.kron(sigmax, sigmax) +\
            yy * np.kron(sigmay, sigmay) +\
            zz * np.kron(sigmaz, sigmaz) +\
            xl * np.kron(sigmax, np.eye(2)) +\
            xr * np.kron(np.eye(2), sigmax) +\
            yl * np.kron(sigmay, np.eye(2)) +\
            yr * np.kron(np.eye(2), sigmay) +\
            zl * np.kron(sigmaz, np.eye(2)) +\
            zr * np.kron(np.eye(2), sigmaz)
        u = expm(-1j * h).reshape((2, 2, 2, 2))
        phi = np.tensordot(u, u.conj(), axes=0)
        phi = phi.transpose((0, 4, 1, 5, 2, 6, 3, 7)).reshape((4, 4, 4, 4))
        rhos_gen = coupled_dynamics(im_exact1, im_exact2, phi, rho_in)
        rhos_trained = coupled_dynamics(im_trained1, im_trained2, phi, rho_in)
        plt.subplot(rows_number, 2, k + 1)
        plt.title(r"$J_x={}, J_y={}, J_z={}$".format(xx, yy, zz), fontdict = {"fontsize": 16})
        plt.plot([np.diag(x) for x in rhos_gen], 'b')
        plt.plot([np.diag(x) for x in rhos_trained], 'b--')
    fig.subplots_adjust(hspace=0.3)
    fig.text(0.5, 0.02, r"${\rm Time}$", ha='center')
    fig.text(0.04, 0.5, r"${\rm diag}(\rho)$", va='center', rotation='vertical')
    fig.show()
    figures.append(fig)

In [ ]:
for name, fig in zip(plotting_config["names"], figures):
    output_path = f"{script_dir}/experiments/output/physical_im/{plotting_config['left_ims'][0]}/{name}.pdf"
    fig.savefig(output_path, bbox_inches = 'tight')